In [2]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin\git\otv\otv\Transfert_Donnees')
import agrege_troncon as at
import Connexion_Transfert as ct
import matplotlib, os, fiona
import geopandas as gp
import pandas as pd
import numpy as np
from datetime import datetime
import Connexion_Transfert as ct
from shapely.wkt import loads
from shapely.ops import nearest_points, polygonize
import Outils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
#importer les odnnées de  base
with ct.ConnexionBdd('local_otv') as c : 
    df = gp.read_postgis("select t.*, v1.cnt nb_intrsct_src, st_astext(v1.geom) as src_geom, v2.cnt as nb_intrsct_tgt, st_astext(v2.geom) as tgt_geom from public.test_agreg_lineaire t left join public.test_agreg_lineaire_vertices_pgr v1 on t.\"source\"=v1.id left join public.test_agreg_lineaire_vertices_pgr v2  on t.target=v2.id", c.connexionPsy)
nature_2_chaussees=['Route à 2 chaussées','Quasi-autoroute','Autoroute']
#donnees generale du jdd
df.set_index('id_ign', inplace=True)  # passer l'id_ign commme index ; necessaire sinon pb avec geometrie vu comme texte
df2_chaussees=df.loc[df.loc[:,'nature'].isin(nature_2_chaussees)] #isoler les troncon de voies decrits par 2 lignes

In [33]:
df_rdpt=at.identifier_rd_pt(df)

In [38]:
df_rdpt

,gid,id,prec_plani,prec_alti,nature,numero,nom_voie_g,nom_voie_d,importance,cl_admin,...,np_ref,id_tronc_e,geom,nb_intrsct_src,src_geom,nb_intrsct_tgt,tgt_geom,id_rdpt,nb_rte_rdpt,nom_rte_rdpt
TRONROUT0000000129676367,100,45963,2.5,2.5,Quasi-autoroute,N137,None,None,1,Nationale,...,None,3119,(LINESTRING (383234.8999999997 6568234.1999999...,2,POINT(383234.9 6568234.19999999),2,POINT(383302.7 6568058.59999999),NaN,NaN,NaN
TRONROUT0000000039535349,8,39694,2.5,2.5,Rd_pt,N537,None,None,3,Nationale,...,None,60,(LINESTRING (375627.4999999997 6570910.6999999...,2,POINT(375627.5 6570910.69999999),3,POINT(375650.2 6570909.49999999),0.0,1.0,N537
TRONROUT0000000032830745,9,125892,10.0,2.5,Quasi-autoroute,N11,None,None,1,Nationale,...,None,962,(LINESTRING (397622.8999999997 6577285.7999999...,2,POINT(397622.9 6577285.79999999),2,POINT(397359.5 6577388.09999999),NaN,NaN,NaN
TRONROUT0000000032830722,10,125871,10.0,2.5,Quasi-autoroute,N11,None,None,1,Nationale,...,None,962,(LINESTRING (397720.6999999997 6577238.5999999...,2,POINT(397720.7 6577238.59999999),2,POINT(397622.9 6577285.79999999),NaN,NaN,NaN
TRONROUT0000000129676495,557,46090,10.0,2.5,Quasi-autoroute,N11,None,None,1,Nationale,...,None,3558,(LINESTRING (382347.8999999997 6571899.9999999...,4,POINT(382347.9 6571899.99999999),3,POINT(382468.5 6571945.39999999),NaN,NaN,NaN
TRONROUT0000000032830721,11,125870,10.0,2.5,Quasi-autoroute,N11,None,None,1,Nationale,...,None,962,(LINESTRING (397720.6999999997 6577238.5999999...,2,POINT(397720.7 6577238.59999999),2,POINT(397752.9 6577223.09999999),NaN,NaN,NaN
TRONROUT0000000129704339,615,46354,2.5,2.5,Route à 1 chaussée,D105,None,None,3,Départementale,...,None,4,(LINESTRING (390249.8999999997 6585946.0999999...,2,POINT(390249.9 6585946.09999999),2,POINT(390342.6 6585938.79999999),NaN,NaN,NaN
TRONROUT0000000032826062,703,122176,2.5,2.5,Route à 1 chaussée,D116,None,None,3,Départementale,...,None,684,(LINESTRING (406730.2999999997 6582333.9999999...,2,POINT(406730.3 6582333.99999999),2,POINT(406670.1 6582504.79999999),NaN,NaN,NaN
TRONROUT0000000129830538,770,46504,5.0,2.5,Route à 2 chaussées,D115,None,None,3,Départementale,...,None,8,(LINESTRING (401056.0999999996 6577431.0999999...,3,POINT(401056.1 6577431.09999999),2,POINT(401072.5 6577362.09999999),NaN,NaN,NaN
TRONROUT0000000032861713,832,150402,2.5,2.5,Route à 1 chaussée,D911,None,None,2,Départementale,...,None,48,(LINESTRING (412267.3999999997 6566203.6999999...,2,POINT(412267.4 6566203.69999999),2,POINT(412296.7 6566249.19999999),NaN,NaN,NaN


In [24]:
toto=list(at.recup_troncon_elementaire('TRONROUT0000000201901354'))

In [25]:
toto.append('TRONROUT0000000201901354')

In [26]:
toto

['TRONROUT0000000201901382',
 'TRONROUT0000000201901385',
 'TRONROUT0000000201901386',
 'TRONROUT0000000201901353',
 'TRONROUT0000000202559715',
 'TRONROUT0000000032976292',
 'TRONROUT0000000032976298',
 'TRONROUT0000000201901354']

In [8]:
at.recup_troncon_parallele_v2(df,toto)

C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees\agrege_troncon.py:184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ligne_filtres['distance']=ligne_filtres.geom.apply(lambda x : gdf_lignes2.centroid.distance(x))


'TRONROUT0000000202559739'

In [ ]:
tata=at.affecter_troncon(df)

In [45]:
df_rd_pt=at.identifier_rd_pt(df)
dico=at.affecter_troncon(df_rd_pt)
with ct.ConnexionBdd('local_otv') as c :
    at.inserer_dico(c, dico)

Index(['gid', 'id', 'prec_plani', 'prec_alti', 'nature', 'numero',
       'nom_voie_g', 'nom_voie_d', 'importance', 'cl_admin', 'gestion',
       'mise_serv', 'it_vert', 'it_europ', 'fictif', 'franchisst', 'largeur',
       'nom_iti', 'nb_voies', 'pos_sol', 'sens', 'alias_g', 'alias_d',
       'inseecom_g', 'inseecom_d', 'codevoie_g', 'codevoie_d', 'codepost_g',
       'codepost_d', 'typ_adres', 'bornedeb_g', 'bornedeb_d', 'bornefin_g',
       'bornefin_d', 'etat', 'z_ini', 'z_fin', 'long_km', 'source', 'target',
       'np_ref', 'id_tronc_e', 'geom', 'nb_intrsct_src', 'src_geom',
       'nb_intrsct_tgt', 'tgt_geom', 'index_right', 'id_rdpt'],
      dtype='object')
0eme occurence : TRONROUT0000000129676367 à 23:00:48 nb ligne traite : 0, nb ligne differente=0


C:\Users\Martin\git\otv\otv\Transfert_Donnees\agrege_troncon.py:229: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ligne_filtres['distance']=ligne_filtres.geom.apply(lambda x : gdf_lignes2.centroid.distance(x))


KeyError: "None of [Float64Index([0.0, nan], dtype='float64')] are in the [index]"

POUR TRAITEMENT DES VOIES A 2 CHAUSSES : TROUVER PARRALLELE

In [172]:
#liste des troncons (parametre entrant)
liste_troncon=toto

In [ ]:
#conversion en geodf
gdf_lignes=gp.GeoDataFrame(df.loc[liste_troncon], geometry='geom')

In [174]:
#union des geometries
gdf_lignes2=gdf_lignes.unary_union

In [176]:
#centroid de la ligne
xmin,ymin,xmax,ymax=gdf_lignes2.interpolate(0.5, normalized=True).buffer(50).bounds

In [177]:
#donnees de base
gdf_global=gp.GeoDataFrame(df, geometry='geom')

In [178]:
#recherche des lignes proches du centroid
lignes_possibles=gdf_global.cx[xmin:xmax,ymin:ymax]

In [ ]:
lignes_possibles

In [180]:
#uniquement les lignes non présentes dans la liste de troncons avec le même nom de voie
ligne_filtres=lignes_possibles.loc[(~lignes_possibles.index.isin(liste_troncon)) & (lignes_possibles['numero']==gdf_lignes.iloc[0]['numero'])]

In [ ]:
#obtenir les distances au centroid
ligne_filtres['distance']=ligne_filtres.geom.apply(lambda x : gdf_lignes2.centroid.distance(x))

In [182]:
#garder uniquement la valeur la plus proche du centroid
ligne_filtres.loc[ligne_filtres['distance']==ligne_filtres['distance'].min()].index[0]

'TRONROUT0000000202559739'

POUR TRAITEMENT DES ROND POINTS

In [4]:
#polygoniser les lignes
#creer la gdf
df.crs={'init':'epsg:2154'}

In [5]:
#creer la lsite des rd points selon le critere atmo aura ; creer une gdf avec la liste
liste_rd_points=[t.buffer(0.1) for t in polygonize(df.geometry) if 12<=((t.length**2)/t.area)<=13] 
gdf_rd_point=gp.GeoDataFrame([i for i in range(len(liste_rd_points))], geometry=liste_rd_points)
gdf_rd_point.crs={'init':'epsg:2154'}
gdf_rd_point.columns=['id_rdpt', 'geometry']    

In [7]:
#jointure spataile pour une gdf avec uniquement les rd_points en lignes avec le numéro
l_dans_p=gp.sjoin(df,gdf_rd_point,op='within')

In [8]:
#lignes qui touchent rd points
#1.ligne qui intersectent avec id_rdpt
l_intersct_rdpt=gp.sjoin(df,l_dans_p.drop('index_right', axis=1), how='inner',op='intersects')
#2.filtre de celle contenue dans le rd points 
l_intersct_rdpt=l_intersct_rdpt.loc[~l_intersct_rdpt.index.isin(l_dans_p.index.tolist())][['id_rdpt','numero_left']]

In [9]:
#trouver le nb de voies qui intersectent chaque rd point et leur noms. renomer les colonnes
carac_rd_pt=(pd.concat([l_intersct_rdpt.groupby('id_rdpt').numero_left.nunique(),
    l_intersct_rdpt.groupby('id_rdpt')['numero_left'].apply(lambda x: ','.join(set(x)))], axis=1))
carac_rd_pt.columns=['nb_rte_rdpt', 'nom_rte_rdpt']

In [11]:
#ajouter l'id_rdpt aux données
gdf_l_rd_point=pd.concat([df,l_dans_p.loc[:,'id_rdpt']],axis=1, sort=False)

In [12]:
#mettre à jour la nature
gdf_l_rd_point['nature']=gdf_l_rd_point.apply(lambda x : 'Rd_pt' if x.id_rdpt>=0 else x['nature'], axis=1)

In [13]:
#ajouter les infos du rd point (nb voies différentes et nom)
gdf_avec_rd_pt=gdf_l_rd_point.merge(carac_rd_pt, how='left',left_on='id_rdpt', right_index=True)

In [67]:
#gdf_avec_rd_pt.loc[gdf_avec_rd_pt.loc[:,'id_rdpt']==l_dans_p['id_rdpt']]
gdf_avec_rd_pt['id_rdpt']==l_dans_p['id_rdpt'].astype('float64')
#l_dans_p.loc[l_dans_p.loc[:,'id_rdpt']].sort_index().sort_index(axis=1)])

ValueError: Can only compare identically-labeled Series objects

In [16]:
#pour export et verif uniquement
gdf_l_rd_point.to_file(r'E:\Boulot\otv\test_linearisation_auto\test2.shp')
gdf_rd_point.to_file(r'E:\Boulot\otv\test_linearisation_auto\test1.shp')
#l_intersct_rdpt.to_file(r'E:\Boulot\otv\test_linearisation_auto\test3.shp')
gdf_avec_rd_pt.to_file(r'E:\Boulot\otv\test_linearisation_auto\test4.shp')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 85: invalid continuation byte

Exception ignored in: 'fiona._env.log_error'
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 85: invalid continuation byte


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 85: invalid continuation byte

Exception ignored in: 'fiona._env.log_error'
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 85: invalid continuation byte


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 86: invalid continuation byte

...

Exception ignored in: 'fiona._env.log_error'
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 85: invalid continuation byte


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 88: invalid continuation byte

Exception ignored in: 'fiona._env.log_error'
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 88: invalid continuation byte


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 88: invalid continuation byte

Exception ignored in: 'fiona._env.log_error'
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 88: invalid continuation byte


In [ ]:
','.join(set(['a','a','b','c']))